# Data Cleaning: Current COVID-19 Data

## Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import timedelta

pd.set_option("display.max_columns", None)
RANDOM_STATE = 42

## Read-In COVID-19 Data
Source: [New York Times COVID-19 Data Hub](https://github.com/nytimes/covid-19-data)

Data for COVID-19 by U.S. County was last updated on August 11, 2020.

In [2]:
covid_df = pd.read_csv('../data/covid_nyt_data.csv')
covid_df.head(2)

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0
1,2020-01-22,Snohomish,Washington,53061.0,1,0


## Read-In 2019 Population Data for States Impacted by Tropical Storms
Source: [United States Census Bureau](https://www.census.gov/data/tables/time-series/demo/popest/2010s-counties-detail.html)

By knowing the population for each county in our the data frame, it allowed us to "standardize" COVID-19 case and death counts. 

#### Population Data

In [3]:
alabama = pd.read_csv('../data/USA_counties_pop/cc-est2019-agesex-01.csv')
dc = pd.read_csv('../data/USA_counties_pop/cc-est2019-agesex-11.csv')
nebraska = pd.read_csv('../data/USA_counties_pop/cc-est2019-agesex-31.csv')
wisconsin = pd.read_csv('../data/USA_counties_pop/cc-est2019-agesex-55.csv')
iowa = pd.read_csv('../data/USA_counties_pop/cc-est2019-agesex-19.csv')
rhode_island = pd.read_csv('../data/USA_counties_pop/cc-est2019-agesex-44.csv')
vermont = pd.read_csv('../data/USA_counties_pop/cc-est2019-agesex-50.csv')
delaware = pd.read_csv('../data/USA_counties_pop/cc-est2019-agesex-10.csv')
connecticut = pd.read_csv('../data/USA_counties_pop/cc-est2019-agesex-09.csv')
kansas = pd.read_csv('../data/USA_counties_pop/cc-est2019-agesex-20.csv')
massachusetts = pd.read_csv('../data/USA_counties_pop/cc-est2019-agesex-25.csv')
new_hampshire = pd.read_csv('../data/USA_counties_pop/cc-est2019-agesex-33.csv')
new_jersey = pd.read_csv('../data/USA_counties_pop/cc-est2019-agesex-34.csv')
michigan = pd.read_csv('../data/USA_counties_pop/cc-est2019-agesex-26.csv')
west_virginia = pd.read_csv('../data/USA_counties_pop/cc-est2019-agesex-54.csv')
maryland = pd.read_csv('../data/USA_counties_pop/cc-est2019-agesex-24.csv')
ohio = pd.read_csv('../data/USA_counties_pop/cc-est2019-agesex-39.csv')
illinois = pd.read_csv('../data/USA_counties_pop/cc-est2019-agesex-17.csv')
oklahoma = pd.read_csv('../data/USA_counties_pop/cc-est2019-agesex-40.csv')
indiana = pd.read_csv('../data/USA_counties_pop/cc-est2019-agesex-18.csv')
missouri = pd.read_csv('../data/USA_counties_pop/cc-est2019-agesex-29.csv')
maine = pd.read_csv('../data/USA_counties_pop/cc-est2019-agesex-23.csv')
new_york = pd.read_csv('../data/USA_counties_pop/cc-est2019-agesex-36.csv')
pennsylvania = pd.read_csv('../data/USA_counties_pop/cc-est2019-agesex-42.csv')
kentucky = pd.read_csv('../data/USA_counties_pop/cc-est2019-agesex-21.csv')
tennessee = pd.read_csv('../data/USA_counties_pop/cc-est2019-agesex-47.csv')
arkansas = pd.read_csv('../data/USA_counties_pop/cc-est2019-agesex-05.csv')
virginia = pd.read_csv('../data/USA_counties_pop/cc-est2019-agesex-51.csv')
south_carolina = pd.read_csv('../data/USA_counties_pop/cc-est2019-agesex-45.csv')
mississippi = pd.read_csv('../data/USA_counties_pop/cc-est2019-agesex-28.csv')
north_carolina = pd.read_csv('../data/USA_counties_pop/cc-est2019-agesex-37.csv')
louisiana = pd.read_csv('../data/USA_counties_pop/cc-est2019-agesex-22.csv')
georgia = pd.read_csv('../data/USA_counties_pop/cc-est2019-agesex-13.csv')
texas = pd.read_csv('../data/USA_counties_pop/cc-est2019-agesex-48.csv')
florida = pd.read_csv('../data/USA_counties_pop/cc-est2019-agesex-12.csv')
new_mexico = pd.read_csv('../data/USA_counties_pop/cc-est2019-agesex-35.csv', encoding='ISO-8859-1')

#### Concatenate Data Frames

In [4]:
county_pop = pd.concat([alabama, dc, nebraska, wisconsin, iowa, rhode_island, vermont, delaware, connecticut, kansas, massachusetts, new_hampshire, new_jersey, michigan, west_virginia, maryland, ohio, illinois, oklahoma, indiana, missouri, maine, new_york, pennsylvania, kentucky, tennessee, arkansas, south_carolina, mississippi, north_carolina, louisiana, georgia, texas, florida, new_mexico, virginia])

In [5]:
county_pop = county_pop[county_pop['YEAR'] == 12][['STNAME', 'CTYNAME', 'POPESTIMATE']]

In [6]:
county_pop.head(1)

,STNAME,CTYNAME,POPESTIMATE
11,Alabama,Autauga County,55869


#### Clean County Names

In [7]:
county_pop['CTYNAME'] = county_pop['CTYNAME'].map(lambda x: x.replace('County', '').strip(' '))
county_pop['CTYNAME'] = county_pop['CTYNAME'].map(lambda x: x.replace('Parish', '').strip(' '))

## Merge COVID-19 Data with 2019 Population Data
***NOTE:*** Only rows that have a history of tropical storms will have population data. This is because we will only be comparing COVID-19 risk in counties that have historical tropical storm data.

In [8]:
covid_df = pd.merge(left = covid_df, right = county_pop, how = 'left', left_on = ['state', 'county'], right_on = ['STNAME', 'CTYNAME'])

In [9]:
covid_df.head(1)

,date,county,state,fips,cases,deaths,STNAME,CTYNAME,POPESTIMATE
0,2020-01-21,Snohomish,Washington,53061.0,1,0,NaN,NaN,NaN


In [10]:
# Drop Unnecessary Columns

covid_df = covid_df.drop(columns = ['STNAME', 'CTYNAME'])

In [11]:
# Rename population column

covid_df = covid_df.rename(columns = {'POPESTIMATE': '2019_population'})

In [12]:
covid_df.head(2)

,date,county,state,fips,cases,deaths,2019_population
0,2020-01-21,Snohomish,Washington,53061.0,1,0,NaN
1,2020-01-22,Snohomish,Washington,53061.0,1,0,NaN


## Create a Column to "Standardize" Case Counts
According to our [research](https://www.fastcompany.com/90529280/what-is-a-covid-19-red-zone-do-you-live-in-one-heres-how-to-find-out), when comparing data by region, cases are represented as number of cases per 100,000 people.

In [13]:
covid_df['cases_per_100000'] = round((covid_df['cases']/covid_df['2019_population']) * 100000, 1)

In [14]:
covid_df.head(2)

,date,county,state,fips,cases,deaths,2019_population,cases_per_100000
0,2020-01-21,Snohomish,Washington,53061.0,1,0,NaN,NaN
1,2020-01-22,Snohomish,Washington,53061.0,1,0,NaN,NaN


## Filter Data Frame to Specific Time Frames
According to an [article](https://www.fastcompany.com/90529280/what-is-a-covid-19-red-zone-do-you-live-in-one-heres-how-to-find-out) published by Fast Company, the White House defines a COVID-19 "red zone" as any state that has experienced 100 or more new cases per 100,000 people in the last week. Therefore, in addition to total number of cases, we will look at the change in cases over the last week.

### Most Recent Date (August 11, 2020)

In [15]:
covid_df['date'] = pd.to_datetime(covid_df['date'])
covid_current = covid_df[covid_df['date'] == covid_df['date'].max()].copy()

In [16]:
covid_current.head(1)

,date,county,state,fips,cases,deaths,2019_population,cases_per_100000
421390,2020-08-11,Autauga,Alabama,1001.0,1215,23,55869.0,2174.7


### Data For Previous Week (August 4, 2020)

In [17]:
covid_one_week_ago = covid_df[covid_df['date'] == (covid_df['date'].max() - timedelta(7))]
covid_one_week_ago = covid_one_week_ago.rename(columns = {'cases':'previous_cases', 'deaths': 'previous_deaths', 'cases_per_100000': 'previous_cases_per_100000'})
covid_one_week_ago = covid_one_week_ago[['date', 'county', 'state', 'previous_cases','previous_deaths', 'previous_cases_per_100000']]

### Calculate Change in Cases Over the Previous Week

#### Merge Data for Previous Week with Most Recent Data

In [18]:
covid_current = pd.merge(left = covid_current, right = covid_one_week_ago, how = 'left', on = ['state', 'county'])

#### Calculate Change in Cases (Raw)

In [19]:
covid_current['change_in_cases'] = covid_current['cases'] - covid_current['previous_cases']

#### Calculate Change in Cases per 100,000 people ("Standardized")

In [20]:
covid_current['change_in_case_ratio'] = covid_current['cases_per_100000'] - covid_current['previous_cases_per_100000']

## Filter Data Frame to Only Include States and Counties with a History of Tropical Storms

In [21]:
def state_mask(state, df = covid_current):
    return df['state'] == state

In [22]:
mask1 = state_mask('Alabama')
mask2 = state_mask('Florida')
mask3 = state_mask('Georgia')
mask4 = state_mask('Louisiana')
mask5 = state_mask('Mississippi')
mask6 = state_mask('Tennessee')
mask7 = state_mask('North Carolina')
mask8 = state_mask('South Carolina')
mask9 = state_mask('Texas')
mask10 = state_mask('Connecticut')
mask11 = state_mask('District of Columbia')
mask12 = state_mask('Delaware')
mask13 = state_mask('Massachusetts')
mask14 = state_mask('Maryland')
mask15 = state_mask('Maine')
mask16 = state_mask('New Hampshire')
mask17 = state_mask('New Jersey')
mask18 = state_mask('New York')
mask19 = state_mask('Ohio')
mask20 = state_mask('Pennsylvania')
mask21 = state_mask('Rhode Island')
mask22 = state_mask('Virginia')
mask23 = state_mask('Vermont')
mask24 = state_mask('West Virginia')
mask25 = state_mask('Illinois')
mask26 = state_mask('Indiana')
mask27 = state_mask('Kentucky')
mask28 = state_mask('Missouri')
mask29 = state_mask('Nebraska')
mask30 = state_mask('New Mexico')
mask31 = state_mask('Wisconsin')
mask32 = state_mask('Iowa')
mask33 = state_mask('Kansas')
mask34 = state_mask('Oklahoma')
mask35 = state_mask('Michigan')
mask36 = state_mask('Arkansas')

In [23]:
covid_hurricane_states = covid_current[mask1|mask2|mask3|mask4|mask5|mask6|mask7|mask8| mask9| mask10| mask11| mask12|mask13| mask14| mask15| mask16|mask17|mask18|mask19|mask20|mask21|mask22|mask23|mask24|mask25|mask26|mask27|mask28|mask29|mask30|mask31|mask32|mask33|mask34|mask35|mask36]

In [24]:
covid_hurricane_states.shape

(2521, 14)

## Export Data to CSV

In [25]:
#covid_hurricane_states.to_csv('../data/covid_hurricane_states.csv', index = False)